<a href="https://colab.research.google.com/github/wubeshetA/ML/blob/main/school-assignments/transfer_learning/Transfer_Learning_Assignment_wubeshet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Vegetable Classification

This goal of this project is to use pretrained models with a new dataset to classify different kind of vegetables.


## Import packages and pre-trained models

In [ ]:
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import classification_report, confusion_matrix
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from google.colab import drive
from tensorflow.keras.applications import VGG16, InceptionV3, VGG19, MobileNet,EfficientNetB2, MobileNetV3Small
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.models import Model

In [ ]:
drive.mount('/content/gdrive')
!unzip "/content/gdrive/My Drive/Colab Notebooks/Vegetables.zip" -d "/content/"


In [ ]:
train_data_dir = '/content/Vegetables/train'

### Data Augmentation Configuration and Training Data Generator Initialization

In [ ]:
train_datagen = ImageDataGenerator(rescale=1.0/255.0, horizontal_flip=True,vertical_flip=True, rotation_range=90)

train_generator = train_datagen.flow_from_directory(
        train_data_dir,
        target_size=(224,224),
        batch_size=64,
        class_mode='categorical',
        shuffle=True)

Found 7000 images belonging to 7 classes.


In [ ]:
train_generator.class_indices

{'Bean': 0,
 'Broccoli': 1,
 'Carrot': 2,
 'Cucumber': 3,
 'Papaya': 4,
 'Potato': 5,
 'Tomato': 6}

## Pretrained Models

In [ ]:
# Load pre-trained models
base_models = [InceptionV3, MobileNetV3Small, MobileNet]
pre_models = []

Initialization of Pre-trained Models and Freezing Layers

In [ ]:
for base_model in base_models:
    base = base_model(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    pre_models.append(base)

for pretrained_model in pre_models:
    for layer in pretrained_model.layers:
        layer.trainable = False

Adding Flatten and Final Dense Layer to Pre-trained Models

In [ ]:
models = []
for pretrained_model in pre_models:
    last_layer = Flatten()(pretrained_model.output)
    final_layer = Dense(7, activation='softmax')(last_layer)

    model = Model(inputs=pretrained_model.input, outputs=final_layer)
    models.append(model)

## Compile models

In [ ]:
for model in models:
  model.compile(optimizer='adam',loss='categorical_crossentropy',metrics='accuracy')

### Summary

In [ ]:
for model in models:
    model.summary()

Model: "model_6"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_7 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 conv2d_188 (Conv2D)         (None, 111, 111, 32)         864       ['input_7[0][0]']             
                                                                                                  
 batch_normalization_188 (B  (None, 111, 111, 32)         96        ['conv2d_188[0][0]']          
 atchNormalization)                                                                               
                                                                                                  
 activation_188 (Activation  (None, 111, 111, 32)         0         ['batch_normalization_18

### Training

In [ ]:
# for model in models:
models[0].fit_generator(train_generator, epochs=4)

<ipython-input-29-e6b5318619c4>:2: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  models[0].fit_generator(train_generator, epochs=4)


Epoch 1/4
110/110 [==============================] - 879s 8s/step - loss: 1.6759 - accuracy: 0.8929
Epoch 2/4
110/110 [==============================] - 864s 8s/step - loss: 0.0802 - accuracy: 0.9847
Epoch 3/4
110/110 [==============================] - 864s 8s/step - loss: 0.0737 - accuracy: 0.9874
Epoch 4/4
110/110 [==============================] - 865s 8s/step - loss: 0.0663 - accuracy: 0.9889


In [ ]:
models[1].fit_generator(train_generator, epochs=4)

<ipython-input-30-26ee0930fcac>:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  models[1].fit_generator(train_generator, epochs=4)


Epoch 1/4
110/110 [==============================] - 187s 2s/step - loss: 2.0184 - accuracy: 0.2706
Epoch 2/4
110/110 [==============================] - 183s 2s/step - loss: 1.5950 - accuracy: 0.3640
Epoch 3/4
110/110 [==============================] - 184s 2s/step - loss: 1.5279 - accuracy: 0.4121
Epoch 4/4
110/110 [==============================] - 184s 2s/step - loss: 1.4029 - accuracy: 0.4730


In [ ]:
models[2].fit_generator(train_generator, epochs=4)

<ipython-input-31-67f67e72f350>:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  models[2].fit_generator(train_generator, epochs=4)


Epoch 1/4
110/110 [==============================] - 412s 4s/step - loss: 0.2871 - accuracy: 0.9593
Epoch 2/4
110/110 [==============================] - 406s 4s/step - loss: 0.0531 - accuracy: 0.9917
Epoch 3/4
110/110 [==============================] - 408s 4s/step - loss: 0.0795 - accuracy: 0.9906
Epoch 4/4
110/110 [==============================] - 410s 4s/step - loss: 0.0543 - accuracy: 0.9926


### Models Evaluation

In [ ]:
#TestData
test_data_dir = '/content/Vegetables/test'
test_datagen = ImageDataGenerator(rescale=1./255)

test_batchsize = 64

test_generator = test_datagen.flow_from_directory(
        test_data_dir,
        target_size=(224,224),
        batch_size=64,
        class_mode='categorical',
        shuffle=False)


Found 1400 images belonging to 7 classes.


In [ ]:
evaluations = []
for model in models:
  evaluations.append(model.evaluate_generator(test_generator))


<ipython-input-35-48f512dded22>:3: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  evaluations.append(model.evaluate_generator(test_generator))


In [ ]:
# Print evaluation results
for i, result in enumerate(evaluations):
    print(f'Model {i+1} - Loss: {result[0]}, Accuracy: {result[1]}')

Model 1 - Loss: 0.061347927898168564, Accuracy: 0.9921428561210632
Model 2 - Loss: 1.3239271640777588, Accuracy: 0.5335714221000671
Model 3 - Loss: 0.032811008393764496, Accuracy: 0.9942857027053833
